# Projet Knowledge Extraction - Partie A : Preprocessing et Représentation Text

**Université Paris Cité - Master 2 VMI**
**Cours :** IFLCE085 Recherche et extraction sémantique à partir de texte (Prof. Salima Benbernou)

**Équipe :**
- **Partie A (Preprocessing) : Jacques Gastebois**
- Partie B : Boutayna EL MOUJAOUID
- Partie C : Franz Dervis
- Partie D : Aya Benkabour

---

## Étape 1 : Setup et Importations
Objectif : Configurer l'environnement et importer les librairies nécessaires.

**Note importante** : Sur Mac ARM avec Python 3.8, spaCy doit être installé via conda (pas pip).

In [ ]:
import sys
# Installation des dépendances de base
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install nltk
!{sys.executable} -m pip install scikit-learn
!{sys.executable} -m pip install transformers
!{sys.executable} -m pip install torch

# Installation de spaCy via conda (nécessaire pour Python 3.8 sur Mac ARM)
!conda install -c conda-forge spacy -y

# Téléchargement du modèle spaCy anglais
!{sys.executable} -m spacy download en_core_web_sm



import os
import json
import pandas as pd
import numpy as np
import nltk
import spacy
import preprocessing as pp # Notre module local

# Configuration de l'affichage pandas
pd.set_option('display.max_colwidth', 100)

print("Environnement configuré avec succès.")

/bin/bash: line 1: conda: command not found
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 88.6 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


ModuleNotFoundError: No module named 'preprocessing'

## Étape 2 : Chargement et Exploration des Données (SciREX)
Objectif : Télécharger (si nécessaire) et charger le dataset SciREX.

In [ ]:
import urllib.request
import tarfile

DATA_DIR = "release_data"
DATA_URL = "https://github.com/allenai/SciREX/raw/master/scirex_dataset/release_data.tar.gz"
TAR_FILE = "release_data.tar.gz"

def download_and_extract_data():
    if not os.path.exists(DATA_DIR):
        print("Dossier de données non trouvé. Téléchargement en cours...")
        try:
            urllib.request.urlretrieve(DATA_URL, TAR_FILE)
            print("Téléchargement terminé. Extraction...")
            with tarfile.open(TAR_FILE, "r:gz") as tar:
                tar.extractall()
            print("Extraction terminée.")
        except Exception as e:
            print(f"Erreur lors du téléchargement/extraction : {e}")
    else:
        print("Les données sont déjà présentes.")

download_and_extract_data()

FILES = {
    "train": os.path.join(DATA_DIR, "train.jsonl"),
    "dev": os.path.join(DATA_DIR, "dev.jsonl"),
    "test": os.path.join(DATA_DIR, "test.jsonl")
}

def load_jsonl(file_path):
    """Charge un fichier JSONL dans une liste de dictionnaires."""
    data = []
    if os.path.exists(file_path):
        with open(file_path, 'r', encoding='utf-8') as f:
            for line in f:
                data.append(json.loads(line))
    else:
        print(f"Attention : Fichier {file_path} introuvable.")
    return data

# Chargement des données
print("Chargement des données...")
train_data = load_jsonl(FILES["train"])
dev_data = load_jsonl(FILES["dev"])
test_data = load_jsonl(FILES["test"])

print(f"Nombre de documents Train : {len(train_data)}")
print(f"Nombre de documents Dev   : {len(dev_data)}")
print(f"Nombre de documents Test  : {len(test_data)}")

In [ ]:
# Exploration d'un document type
if train_data:
    doc_example = train_data[0]
    print("\nClés disponibles dans un document :")
    print(list(doc_example.keys()))

    print("\nExemple de contenu (champs principaux) :")
    print(f"ID: {doc_example.get('doc_id')}")
    if 'words' in doc_example:
        print(f"Début du texte (50 premiers mots) : {' '.join(doc_example['words'][:50])}...")
